# Production of alpha-tocopherol (vitamin E) in E. coli

## 1. Introduction

### 1.1 Literature review of the compound (<500 words) Daniel

### 1.2 Literature review of the cell factory (<500 words) Ben

## 2. Problem definition (<300 words) Kaare

What is it that you want to achieve or improve? Examples could be:

- Product-related problems:
    - Low production rates
    - Low concentrations
    - Low yields
- Host cell-related problems:
    - Slow growth
    - Toxic properties
    - Adding new substrates
    - Byproduct formation

## 3. *If Project category II:* Selection and assessment of existing GSM (<500 words) Joachim

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References